In [103]:
### Connect to Google sheets... read from google sheets
### Write straight to Google sheets
### Automatically download from Dukascopy


In [104]:
import pandas as pd
import os, re
from datetime import timedelta
import numpy as np

In [105]:
stats_path = '../Downloads/LQD Stats - 2020 Week 25 - Stats.csv'

In [110]:
d = pd.read_csv(stats_path)
d.tail()

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price.1,Commission,Taxes,Swap,Profit,Balance,Percent,MAE,MFE,Tags
22,52534655,2020.06.19 16:25:12,buy,1.57,audjpy,73.371,0.0,0.0,2020.06.19 16:41:23,73.279,-10.99,0.0,0.0,-135.08,"$2,739.13",-4.93%,NaN,NaN,NaN
23,52535103,2020.06.19 16:44:53,buy,3,audjpy,73.294,0.0,0.0,2020.06.19 17:00:28,73.274,-21,0.0,0.0,-56.13,"$2,683.00",-2.09%,NaN,NaN,NaN
24,52535388,2020.06.19 17:00:34,sell,3,audjpy,73.269,73.5,0.0,2020.06.19 17:04:08,73.275,-21,0.0,0.0,-16.84,"$2,666.16",-0.63%,NaN,NaN,NaN
25,52535656,2020.06.19 17:17:02,buy,3,audjpy,73.104,0.0,0.0,2020.06.19 20:19:25,73.252,-21,0.0,0.0,415.38,"$3,081.54",13.48%,NaN,NaN,NaN
26,52535802,2020.06.19 17:20:52,buy,3,audjpy,73.079,0.0,0.0,2020.06.19 20:19:21,73.252,-21,0.0,0.0,485.54,"$3,567.08",13.61%,NaN,NaN,NaN


In [111]:
end_date = '13.06'
[x for x in os.listdir('../Downloads') if 'Candlestick' in x]

['AUDJPY_Candlestick_1_M_BID_01.06.2020-07.06.2020.csv',
 'AUDJPY_Candlestick_1_M_BID_05.06.2020-12.06.2020.csv',
 'AUDJPY_Candlestick_1_M_BID_13.06.2020-19.06.2020.csv',
 'AUDJPY_Candlestick_1_M_BID_30.04.2019-08.05.2020.csv',
 'AUS.IDXAUD_Candlestick_1_M_BID_01.09.2019-10.04.2020.csv',
 'AUS.IDXAUD_Candlestick_1_M_BID_05.04.2020-16.04.2020.csv',
 'de30_DEU.IDXEUR_Candlestick_1_M_BID_13.06.2020-19.06.2020.csv',
 'DEU.IDXEUR_Candlestick_1_M_BID_01.09.2019-10.04.2020.csv',
 'GBPAUD_Candlestick_1_M_BID_01.06.2020-07.06.2020.csv',
 'GBPAUD_Candlestick_1_M_BID_05.06.2020-12.06.2020.csv',
 'GBPAUD_Candlestick_1_M_BID_13.06.2020-19.06.2020.csv',
 'GBPAUD_Candlestick_1_M_BID_30.04.2019-08.05.2020.csv',
 'GBPCAD_Candlestick_1_M_BID_01.06.2020-07.06.2020.csv',
 'GBPCAD_Candlestick_1_M_BID_05.06.2020-12.06.2020.csv',
 'GBPJPY_Candlestick_1_M_BID_05.06.2020-12.06.2020.csv',
 'GBPNZD_Candlestick_1_M_BID_01.06.2020-07.06.2020.csv',
 'GBPNZD_Candlestick_1_M_BID_05.06.2020-12.06.2020.csv',
 'GBPUSD_C

In [112]:
# find data in downloads
data = []
for x in os.listdir('../Downloads/'):
    for y in set(d['Item'].dropna()):
        if re.match(y.upper(), x.upper()) and end_date in x:
            data.append(x)
print(data)

p = {
    x.split('_')[0].upper(): pd.read_csv('../Downloads/' + x)
    for x in data
}

for x in p.values():
    x['time'] = pd.to_datetime(x['Local time'].apply(lambda x: x.replace(' GMT', '')), format='%d.%m.%Y %H:%M:%S.%f%z')
    x['gmt+1'] = x[['time']].set_index('time').tz_convert('Etc/GMT-1').index

['AUDJPY_Candlestick_1_M_BID_13.06.2020-19.06.2020.csv', 'de30_DEU.IDXEUR_Candlestick_1_M_BID_13.06.2020-19.06.2020.csv', 'GBPAUD_Candlestick_1_M_BID_13.06.2020-19.06.2020.csv']


In [113]:
p.keys()

dict_keys(['AUDJPY', 'DE30', 'GBPAUD'])

In [114]:
d['open time dt'] = pd.to_datetime(d['Open Time'].apply(lambda x: x + '+0100' if x is not np.nan else np.nan))
d['close time dt'] = pd.to_datetime(d['Close Time'].apply(lambda x: x + '+0100' if x is not np.nan else np.nan))

In [115]:
# # find start time, end time
# d.loc[d['Item'] == 'gbpcad']
# d.loc[d['Item'] == 'gbpcad', ['Open Time', 'Close Time']]
# d.loc[d['Item'] == 'gbpcad', 'Open Time'][4]

In [116]:
# find closest match in p
# d.dtypes

In [117]:
# d['open time dt'] = pd.to_datetime(d['Open Time'], utc=True)
# d[['open time dt']].set_index('open time dt').tz_convert('Etc/GMT+1').apply(lambda x: x + timedelta(hours=1))
# d['open time dt'].replace(tzinfo='UTC')

In [118]:
# d.loc[d['Item'] == 'gbpcad', 'open time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'open time dt'][4].second)

In [119]:
# d.loc[d['Item'] == 'gbpaud', [x for x in d.columns if x not in ['Commission', 'Taxes', 'Swap', 'MAE', 'MFE', 'Tags', 'S / L', 'Balance', 'T / P', 'Percent']]]

In [120]:
# # open
# p.loc[p['gmt+1'] == d.loc[d['Item'] == 'gbpcad', 'open time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'open time dt'][4].second)]

In [121]:
# # close
# p.loc[p['gmt+1'] == d.loc[d['Item'] == 'gbpcad', 'close time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'close time dt'][4].second)]

In [122]:
# # range of trade
# p.loc[
#     (p['gmt+1'] >= d.loc[d['Item'] == 'gbpcad', 'open time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'open time dt'][4].second)) &
#     (p['gmt+1'] < d.loc[d['Item'] == 'gbpcad', 'close time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'close time dt'][4].second))
# ]

In [123]:
# p.loc[
#     (p['gmt+1'] >= d.loc[d['Item'] == 'gbpcad', 'open time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'open time dt'][4].second)) &
#     (p['gmt+1'] < d.loc[d['Item'] == 'gbpcad', 'close time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'close time dt'][4].second)),
#     'High'
# ].max()

In [124]:
# p.loc[
#     (p['gmt+1'] >= d.loc[d['Item'] == 'gbpcad', 'open time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'open time dt'][4].second)) &
#     (p['gmt+1'] < d.loc[d['Item'] == 'gbpcad', 'close time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'close time dt'][4].second)),
#     ['High', 'Low']
# ].agg({'High': 'max', 'Low': 'min'})

In [125]:
# d.loc[d['Item'] == 'gbpcad', 'Price'][4], d.loc[d['Item'] == 'gbpcad', 'Price.1'][4]

In [126]:
# needs to check buy or sell

In [127]:
# d.loc[d['Item'] == 'gbpcad', 'Price'][4] - p.loc[
#     (p['gmt+1'] >= d.loc[d['Item'] == 'gbpcad', 'open time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'open time dt'][4].second)) &
#     (p['gmt+1'] < d.loc[d['Item'] == 'gbpcad', 'close time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'close time dt'][4].second)),
#     ['High', 'Low']
# ].agg({'High': 'max', 'Low': 'min'})['Low']

In [128]:
# p.loc[
#     (p['gmt+1'] >= d.loc[d['Item'] == 'gbpcad', 'open time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'open time dt'][4].second)) &
#     (p['gmt+1'] < d.loc[d['Item'] == 'gbpcad', 'close time dt'][4] - timedelta(seconds=d.loc[d['Item'] == 'gbpcad', 'close time dt'][4].second)),
#     ['High', 'Low']
# ].agg({'High': 'max', 'Low': 'min'})['High'] - d.loc[d['Item'] == 'gbpcad', 'Price'][4]

In [129]:
###
### see if you left money on the table in the day ###
###

In [130]:
# read in trade (symbol, open time, close time, open price)
def get_mae(row):
    
    try:
        r = p[row['Item'].upper()]
    except Exception as e:
        print('E:', e)
        return None, None
    
    r = r.loc[
        (r['gmt+1'] >= row['open time dt'] - timedelta(seconds=row['open time dt'].second)) &
        (r['gmt+1'] < row['close time dt'] - timedelta(seconds=row['close time dt'].second)),
        ['High', 'Low']
    ].agg({'High': 'max', 'Low': 'min'}).to_dict()
    
    if row['Type'] == 'buy':
        mae = -(row['Price'] - r['Low'])
        mfe = r['High'] - row['Price']
    if row['Type'] == 'sell':
        mae = -(r['High'] - row['Price'])
        mfe = row['Price'] - r['Low']
    else:
        pass
    
    return mae, mfe

# get mfe, mae

In [131]:
d = d.loc[d['Type'].isin(['buy', 'sell'])]

In [132]:
d['mae'] = d.apply(lambda x: get_mae(x)[0] if x is not np.nan else np.nan, axis=1)
d['mfe'] = d.apply(lambda x: get_mae(x)[1] if x is not np.nan else np.nan, axis=1)

In [133]:
def diff(row):
    if row['Type'] == 'buy':
        return row['Price.1'] - row['Price']
    if row['Type'] == 'sell':
        return row['Price'] - row['Price.1']

In [134]:
d['diff'] = d.apply(lambda x: diff(x), axis=1)

In [135]:
# pct of mfe is what you actually took
d['mfe%'] = d['diff'] / d['mfe']
# what pct did you take out of the total mae + mfe?
d['mae%'] = d['diff'] / (abs(d['mae']) + d['mfe'])
# what % of trade was dd?
d['dd%'] = d['mae'] / (abs(d['mae']) + d['mfe'])

In [136]:
def clean(row):
    if row['mfe'] < 0:
        row['mfe%'] = '-'
        row['mae%'] = '-'
    else:
        row['mfe%'] = round(row['mfe%'], 2)
        row['mae%'] = round(row['mae%'], 2)
    row['dd%'] = round(row['dd%'], 2)
    return row

In [137]:
d = d.apply(lambda x: clean(x), axis=1)

In [138]:
d[[x for x in d.columns if x not in ['open time dt', 'close time dt', 'Commission', 'Taxes', 'Swap', 'MAE', 'MFE', 'Tags', 'S / L', 'Balance', 'T / P', 'Percent']]]

,Ticket,Open Time,Type,Size,Item,Price,Close Time,Price.1,Profit,mae,mfe,diff,mfe%,mae%,dd%
0,52367002,2020.06.15 02:43:02,buy,0.5,gbpaud,1.83122,2020.06.15 02:48:25,1.82964,-54.04,-0.00156,-0.00028,-0.00158,-,-,-1.22
1,52421492,2020.06.16 13:14:35,buy,0.01,de30,12329.52000,2020.06.16 14:06:00,12361.13000,8.92,-4.46000,77.08000,31.61000,0.41,0.39,-0.05
2,52433638,2020.06.16 16:01:57,buy,1,audjpy,73.42200,2020.06.16 16:08:03,73.60300,168.56,-0.10000,0.28400,0.18100,0.64,0.47,-0.26
4,52446251,2020.06.17 02:13:30,buy,0.5,audjpy,73.78100,2020.06.17 02:14:52,73.72900,-24.23,-0.01700,0.00700,-0.05200,-7.43,-2.17,-0.71
5,52446254,2020.06.17 02:13:10,buy,0.5,audjpy,73.79900,2020.06.17 02:14:19,73.74900,-23.30,-0.03500,-0.01100,-0.05000,-,-,-1.46
6,52446268,2020.06.17 02:15:01,buy,0.5,audjpy,73.74400,2020.06.17 03:32:27,73.50800,-110.02,-0.22100,0.02500,-0.23600,-9.44,-0.96,-0.90
7,52446277,2020.06.17 02:15:19,buy,0.5,audjpy,73.71000,2020.06.17 03:32:27,73.50800,-94.16,-0.18700,0.05900,-0.20200,-3.42,-0.82,-0.76
8,52455110,2020.06.17 09:32:14,buy,0.01,de30,12427.83000,2020.06.17 09:36:48,12396.88000,-8.72,-14.73000,4.73000,-30.95000,-6.54,-1.59,-0.76
11,52502118,2020.06.18 14:50:50,buy,0.5,gbpaud,1.81385,2020.06.18 14:54:07,1.81499,39.02,-0.00120,0.00180,0.00114,0.63,0.38,-0.40
12,52502497,2020.06.18 14:57:15,buy,0.5,gbpaud,1.81614,2020.06.18 15:00:20,1.81550,-21.90,-0.00037,0.00093,-0.00064,-0.69,-0.49,-0.28


In [139]:
# d[[x for x in d.columns if x not in ['open time dt', 'close time dt', 'Commission', 'Taxes', 'Swap', 'MAE', 'MFE', 'Tags', 'S / L', 'Balance', 'T / P', 'Percent']]]

In [140]:
# what's the max the trade goes, after closing, before it would have stopped you out?


In [141]:
# for row

# find price for row['Item'] at time row['close time']

# after that point, did it hit the stop loss? 
    # (if it was already stopped out, n/a)
    # when did it hit the stop loss?
    # if it never did, you are a cunt
    # if it did eventually, how far did it run first?
    
# time before stop out
# distance before reversing



In [142]:
# d.loc[d['Item'] == 'gbpcad', 'close time dt'][22]

In [143]:
# p['GBPCAD'].loc[p['GBPCAD']['gmt+1'] >= d.loc[d['Item'] == 'gbpcad', 'close time dt'][22]]

In [144]:
# p['GBPCAD'].loc[p['GBPCAD']['gmt+1'] >= d.loc[d['Item'] == 'gbpcad', 'close time dt'][22], ['Low', 'High']].agg({'Low': 'min', 'High': 'max'})

In [145]:
def mlot(row):
    
    order = row['Type']
    close = row['Price.1']
    sl = row['S / L']
    low = p[row['Item'].upper()].loc[p[row['Item'].upper()]['gmt+1'] >= row['close time dt'], ['Low', 'High']].agg({'Low': 'min', 'High': 'max'})['Low']
    high = p[row['Item'].upper()].loc[p[row['Item'].upper()]['gmt+1'] >= row['close time dt'], ['Low', 'High']].agg({'Low': 'min', 'High': 'max'})['High']

    if order == 'buy':
        #print('money on table:', high - close)
        row['mlot'] = high-close
        if low < sl:
            #print('dist after sl:', sl - low)
            row['dasl'] = sl-low
        else:
            #print('no stop out')
            row['dasl'] = None
    else:
        #print('money on table:', close-low)
        row['mlot'] = close-low
        if high > sl:
            row['dasl'] = high-sl
            #print('dist after sl:', high - sl)
        else:
            row['dasl'] = None
            #print('no stop out')
    
    return row

In [146]:
d = d.apply(lambda x: mlot(x), axis=1)

In [147]:
# order = d.loc[d['Item'] == 'gbpcad', 'Type'][22]
# close = d.loc[d['Item'] == 'gbpcad', 'Price.1'][22]
# sl = d.loc[d['Item'] == 'gbpcad', 'S / L'][22]
# low = p['GBPCAD'].loc[p['GBPCAD']['gmt+1'] >= d.loc[d['Item'] == 'gbpcad', 'close time dt'][22], ['Low', 'High']].agg({'Low': 'min', 'High': 'max'})['Low']
# high = p['GBPCAD'].loc[p['GBPCAD']['gmt+1'] >= d.loc[d['Item'] == 'gbpcad', 'close time dt'][22], ['Low', 'High']].agg({'Low': 'min', 'High': 'max'})['High']

In [148]:
# order, close, sl, low, high

In [149]:
# if order == 'buy':
#     print('money on table:', high - close)
#     if low < sl:
#         print('dist after sl:', sl - low)
#     else:
#         print('no stop out')
# else:
#     print('money on table:', close-low)
#     if high > sl:
#         print('dist after sl:', high - sl)
#     else:
#         print('no stop out')
    

In [150]:
def pips(row, item_list):
    if row['Item'] == 'de30':
        pass
    elif row['Item'][-3:] == 'jpy':
        for x in item_list:
            row[x] = row[x]*100
    elif row['Item'][:3] == 'xau':
        pass
    else:
        for x in item_list:
            row[x] = row[x]*10000
    return row

In [151]:
d = d.apply(lambda x: pips(x, ['mae', 'mfe', 'diff', 'dasl', 'mlot']), axis=1)

In [153]:
d[[x for x in d.columns if x not in [
    'Commission', 'Taxes', 'Swap', 'MAE', 'MFE', 'Tags', 'open time dt', 'close time dt', 
    'S / L', 'Balance', 'T / P', 'Percent'
]]]

,Ticket,Open Time,Type,Size,Item,Price,Close Time,Price.1,Profit,mae,mfe,diff,mfe%,mae%,dd%,mlot,dasl
0,52367002,2020.06.15 02:43:02,buy,0.5,gbpaud,1.83122,2020.06.15 02:48:25,1.82964,-54.04,-15.60,-2.80,-15.80,-,-,-1.22,128.00,351.4
1,52421492,2020.06.16 13:14:35,buy,0.01,de30,12329.52000,2020.06.16 14:06:00,12361.13000,8.92,-4.46,77.08,31.61,0.41,0.39,-0.05,123.96,75.6
2,52433638,2020.06.16 16:01:57,buy,1,audjpy,73.42200,2020.06.16 16:08:03,73.60300,168.56,-10.00,28.40,18.10,0.64,0.47,-0.26,70.50,2.0
4,52446251,2020.06.17 02:13:30,buy,0.5,audjpy,73.78100,2020.06.17 02:14:52,73.72900,-24.23,-1.70,0.70,-5.20,-7.43,-2.17,-0.71,57.90,75.1
5,52446254,2020.06.17 02:13:10,buy,0.5,audjpy,73.79900,2020.06.17 02:14:19,73.74900,-23.30,-3.50,-1.10,-5.00,-,-,-1.46,55.90,76.9
6,52446268,2020.06.17 02:15:01,buy,0.5,audjpy,73.74400,2020.06.17 03:32:27,73.50800,-110.02,-22.10,2.50,-23.60,-9.44,-0.96,-0.90,80.00,53.8
7,52446277,2020.06.17 02:15:19,buy,0.5,audjpy,73.71000,2020.06.17 03:32:27,73.50800,-94.16,-18.70,5.90,-20.20,-3.42,-0.82,-0.76,80.00,53.3
8,52455110,2020.06.17 09:32:14,buy,0.01,de30,12427.83000,2020.06.17 09:36:48,12396.88000,-8.72,-14.73,4.73,-30.95,-6.54,-1.59,-0.76,88.21,240.6
11,52502118,2020.06.18 14:50:50,buy,0.5,gbpaud,1.81385,2020.06.18 14:54:07,1.81499,39.02,-12.00,18.00,11.40,0.63,0.38,-0.40,20.80,178.3
12,52502497,2020.06.18 14:57:15,buy,0.5,gbpaud,1.81614,2020.06.18 15:00:20,1.81550,-21.90,-3.70,9.30,-6.40,-0.69,-0.49,-0.28,2.60,185.6
